# Hyperparameter Search for VAECF

This notebook describes how to perform hyperparameter searches in Cornac. As a running example we consider the VAECF model and MovieLens 100K dataset. 

## Setup

In [1]:
import numpy as np
import cornac
from cornac.datasets import movielens
from cornac.eval_methods import RatioSplit
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch

## Prepare an experiment

First, we load our data and instantiate the necessary classes for running an experiment.

In [18]:
# Load MovieLens 100K ratings
ml_100k = movielens.load_feedback(variant="100K")

# Define an evaluation method to split feedback into train, validation and test sets
ratio_split = RatioSplit(data=ml_100k, test_size=0.1, val_size=0.1, seed=123, verbose=False)

# Instantiate Recall@100 and NDCG@100 for evaluation
rec100 = cornac.metrics.Recall(100)
ndcg100 = cornac.metrics.NDCG(100)

# Instantiate VACF with fixed hyperparameters
vaecf = cornac.models.VAECF(k=20, h=40, learning_rate=0.005, n_epochs=100, seed=123, verbose=False)

## Perform searches for the *beta* parameter

Assume for now we are interested in determining a good value for the hyperparameter $\beta$, the weight of the KL term in VAECF's objective:

$$ \mathcal{L}(\theta,\phi) = \mathbb{E}_{q_{\phi}(z|r)}[\log{p_{\theta}(r|z)}] - \beta \cdot KL(q_{\phi}(z|r)||p(z)), $$

where $z$ is the latent variable (user representation), and $r$ is the observed user-item feedback.

### Wrap vaecf into search methods
All we need to do is to wrap our instantiated model ``vaecf`` into a search method and specify the search space for *beta*, the metric of interest, as well as the evaluation method. Cornac supports two types of search methods, namely random and grid searches, we consider both of them here.

In [9]:
# GridSearch
gs_vaecf = GridSearch(
    model=vaecf,
    space=[
        Discrete("beta", [0.0, 0.2, 0.4, 0.6, 0.8, 1., 1.2, 1.4, 1.6, 1.8, 2.]),
    ],
    metric=rec100,
    eval_method=ratio_split,
)

# RandomSearch
rs_vaecf = RandomSearch(
    model=vaecf,
    space=[
        Continuous("beta", low=0.0, high=2.),
    ],
    metric=rec100,
    eval_method=ratio_split,
    n_trails=20,
)

As evident from above, there are two ways we can specify search spaces, namely `Discrete` and `Continuous`. In the latter case, `n_trails` values will be uniformly sampled in the interval [low,high). More details in the [documentation](https://cornac.readthedocs.io/en/latest/hyperopt.html).

### Run an experiment
Next, we put everything into an experiment and run it. The test results corresponding to the best value of *beta* (as measured on the validation set in terms of Recall@100) will be displayed. One can print out more information during this step by setting `verbose=True` when instantiating `VAECF`.

In [10]:
cornac.Experiment(
    eval_method=ratio_split,
    models=[gs_vaecf, rs_vaecf],
    metrics=[rec100],
    user_based=False,
).run()


[GridSearch_VAECF] Training started!

[GridSearch_VAECF] Evaluation started!


100%|███████████████████████████████████████████████████████████████████████████████| 929/929 [00:01<00:00, 839.25it/s]



[RandomSearch_VAECF] Training started!

[RandomSearch_VAECF] Evaluation started!


100%|███████████████████████████████████████████████████████████████████████████████| 929/929 [00:00<00:00, 961.13it/s]



                   | Recall@100 | Train (s) | Test (s)
------------------ + ---------- + --------- + --------
GridSearch_VAECF   |     0.5576 |  105.9125 |   1.1110
RandomSearch_VAECF |     0.5567 |  192.1776 |   0.9696



The best *beta* values found by our search methods are as follows,

In [12]:
print('Grid search: ',gs_vaecf.best_params)
print('Random search: ',rs_vaecf.best_params)

Grid search:  {'beta': 1.4}
Random search:  {'beta': 1.4580994147680832}


It is also possible to access the best model through the attribute `best_model`.

## Perform searches for multiple parameters

We can also perform a joint search for multiple parameters. For instance, in addition to *beta*, lets include the number of epochs. 

In [13]:
# Instantiate VACF with fixed hyperparameters
vaecf = cornac.models.VAECF(k=20, h=40, learning_rate=0.005, seed=123, verbose=False)

# GridSearch
gs_vaecf = GridSearch(
    model=vaecf,
    space=[
        Discrete("n_epochs", [20, 50, 100]),
        Discrete("beta", [0.0, 0.4, 0.8, 1., 1.4, 1.8, 2.]),
    ],
    metric=rec100,
    eval_method=ratio_split,
)

# RandomSearch
rs_vaecf = RandomSearch(
    model=vaecf,
    space=[
        Discrete("n_epochs", [20, 50, 100]),
        Continuous("beta", low=0.0, high=2.),
    ],
    metric=rec100,
    eval_method=ratio_split,
    n_trails=20,
)

# Put everything into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[gs_vaecf, rs_vaecf],
    metrics=[rec100],
    user_based=False,
).run()


[GridSearch_VAECF] Training started!

[GridSearch_VAECF] Evaluation started!


100%|███████████████████████████████████████████████████████████████████████████████| 929/929 [00:00<00:00, 986.72it/s]



[RandomSearch_VAECF] Training started!

[RandomSearch_VAECF] Evaluation started!


100%|███████████████████████████████████████████████████████████████████████████████| 929/929 [00:00<00:00, 932.12it/s]



                   | Recall@100 | Train (s) | Test (s)
------------------ + ---------- + --------- + --------
GridSearch_VAECF   |     0.5666 |  125.2540 |   0.9445
RandomSearch_VAECF |     0.5656 |  135.4239 |   0.9987



The results of our searches are:

In [17]:
print('Grid search: ',gs_vaecf.best_params)
print('Random search: ',rs_vaecf.best_params)

Grid search:  {'beta': 1.0, 'n_epochs': 50}
Random search:  {'beta': 0.8218487454180379, 'n_epochs': 50}


The output of this experiment is different from the previous one, due to a "correlation" between the effect of the considered parameters on training. Recall that *beta* can be thought of as a regularization coefficient controlling the effect of the KL term in VAECF. Previously, we fixed the number of epochs to 100, and more regularization seemed to be necessary to avoid overfitting (our searches selected higher beta values). However, the latter case reveals that we may achieve competitive performance with a smaller *beta* if we reduce the number of training iterations.   